In [ ]:
# test (miniconda3 (Python 3.12.2), open integrated terminal)

In [1]:
import csv

In [2]:
# format prices into lists
close_prices = []
data = []
open_prices = []

with open('./data/prices/csvs/AAPL.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    for row in reader:
        data.append(float(row[1]))

data.reverse()
print(data)

for i in range(0,10):
    if data[i+5] > data[i+4]:
        true_trend = 'up'
    else:
        true_trend = 'down'
#     print(close_prices[i:i+5], close_prices[i+5], m)


[39.48, 35.55, 37.07, 36.98, 37.69, 38.33, 38.45, 38.07, 37.5, 38.27, 38.74, 38.97, 39.21, 38.33, 38.48, 38.18, 39.44, 39.08, 38.67, 41.31, 41.61, 41.63, 42.81, 43.55, 43.56, 42.74, 42.6, 42.36, 42.72, 42.55, 42.7, 42.61, 42.73, 43.01, 42.77, 43.24, 43.56, 43.58, 43.72, 43.29, 43.74, 43.96, 43.88, 43.63, 43.13, 43.23, 44.73, 45.23, 45.43, 45.93, 46.53, 47.01, 46.63, 47.04, 48.77, 47.76, 47.19, 46.7, 47.12, 47.18, 47.49, 47.81, 48.51, 48.84, 48.92, 49.25, 50.03, 49.88, 50.16, 49.74, 49.72, 49.81, 49.81, 50.78, 50.97, 51.13, 51.87, 51.79, 51.32, 51.08, 51.15, 50.17, 52.63, 52.29, 52.94, 52.12, 50.72, 50.73, 50.18, 49.3, 46.43, 47.17, 47.73, 47.52, 47.25, 45.77, 46.65, 45.7, 44.92, 44.74, 44.56, 44.35, 44.58, 43.77, 43.33, 44.91, 45.64, 46.31, 47.54, 48.15, 48.7, 48.55, 48.54, 48.19, 48.47, 49.61, 49.47, 49.87, 49.7, 49.65, 48.89, 49.95, 49.94, 49.48, 50.39, 50.68, 51.1, 51.06, 50.01, 50.31, 50.81, 50.44, 50.83, 51.3, 51.13, 50.84, 51.42, 50.65, 51.81, 52.21, 52.17, 51.76, 51.94, 52.42, 5

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data, columns=['Close Price'])

df['Date'] = pd.date_range(start='2023-07-01', periods=len(data), freq='D')

df['Moving Average'] = df['Close Price'].rolling(window=21).mean()

plt.figure(figsize=(10, 5))
plt.plot(df['Date'], df['Close Price'], label='Close Price', marker='.', markersize=4, linewidth=1)
plt.plot(df['Date'], df['Moving Average'], label='21-Day Moving Average', linestyle='--', linewidth=1)

plt.title('Close Price and Moving Average')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()

plt.grid(True)
plt.show()


In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [12]:
# openai llm without rag, only stock prices

import os
from dotenv import load_dotenv
import openai

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
load_dotenv()

completion = openai.chat.completions.create(
    model="gpt-4",
    messages=[
        {
            "role": "user",
            "content": "",
            "content": "2+2=",
            # "content": "these are the stock prices (closing price) from past 5 days: [37.07, 36.98, 37.69, 38.33, 38.45], last element of the list is the latest closing price. these are the news headlines relating to the stock's company: ['may have reached its all time high', 'is the stock price going to fall?']. predict if the stock price next day will go up or down. your answer should either be up or down, do not elaborate.",
            # "content": "these are the stock prices (closing price) from past 5 days: [37.07, 36.98, 37.69, 38.33, 38.45], last element of the list is the latest closing price. predict if the stock price next day will go up or down. your answer should either be up or down, do not elaborate.", # here i could iteratively pass stock prices?
        },
    ],
)
print(completion.choices[0].message.content)


4


In [16]:
data = []
with open('./data/prices/csvs/AAPL.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip header
    for row in reader:
        data.append(float(row[1]))
        

data.reverse() #from oldest to most recent

sequences = [data[i:i+5] for i in range(10)] #len(data) - 5)] only 10 for now
actual_trends = ['up' if data[i+5] > data[i+4] else 'down' for i in range(len(data) - 5)]

def get_model_prediction(prices):
    
    completion = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {
                "role": "user",
                "content": f"Predict the stock trend based on these prices: {prices}. Your answer should either be 'up' or 'down', do not elaborate.",
            },
        ],
    )

    return completion.choices[0].message.content #completion.choices[0].text.strip()

accurate_values = 0
for prices, true_trend in zip(sequences[:10], actual_trends[:10]):  # Limit to 10 calls as per your example
    predicted_trend = get_model_prediction(prices)
    # print(predicted_trend)
    if predicted_trend.lower() == true_trend:
        accurate_values += 1

print(f"Accuracy: {accurate_values/10:.2%}")

# notes: set temperature to 0 bc accuracy differs every time, or average over multiple tests?



up
up
up
up
down
up
up
up
up
up
Accuracy: 60.00%


In [21]:
def get_model_prediction(prices):
    
    completion = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {
                "role": "user",
                "content": f"Predict the stock trend based on these prices: {prices}. Your answer should either be 'up' or 'down', do not elaborate.",
            },
        ],
    )

    return completion.choices[0].message.content

directory_path = './data/prices/csvs/'
files = os.listdir(directory_path)

for file in files:
    if file.endswith('.csv'):
        data = []
        with open(os.path.join(directory_path, file), newline='') as csvfile:
            print(file)
            reader = csv.reader(csvfile)
            next(reader)
            for row in reader:
                data.append(float(row[1]))

        data.reverse()

        sequences = [data[i:i+5] for i in range(10)]
        actual_trends = ['up' if data[i+5] > data[i+4] else 'down' for i in range(10)]

        accurate_values = 0
        for prices, true_trend in zip(sequences[:10], actual_trends[:10]): 
            predicted_trend = get_model_prediction(prices)
            if predicted_trend.lower() == true_trend:
                accurate_values += 1

        # Print the results for each file
        print(f"File: {file}, Accuracy: {accurate_values/10:.2%}")

EQIX.csv
File: EQIX.csv, Accuracy: 70.00%
FCX.csv
File: FCX.csv, Accuracy: 60.00%
BAC.csv
File: BAC.csv, Accuracy: 70.00%
PEP.csv
File: PEP.csv, Accuracy: 40.00%
LLY.csv
File: LLY.csv, Accuracy: 40.00%
MRK.csv
File: MRK.csv, Accuracy: 40.00%
UNP.csv
File: UNP.csv, Accuracy: 40.00%
ABBV.csv
File: ABBV.csv, Accuracy: 20.00%
ECL.csv
File: ECL.csv, Accuracy: 50.00%
AMT.csv
File: AMT.csv, Accuracy: 70.00%
PG.csv


KeyboardInterrupt: 